In [ ]:
#Imports -----------------------------------------------------------------------------
#Pandas: Reading and analyzing data
import pandas as pd
#Numerical calcuations
import numpy as np
#statistical data visualization
import seaborn as sns
#Use Dates in Datetime Format
import datetime
#Tensorflow
#Evaluate models
import math
#Callback to logg model fitting time
import time
from timeit import default_timer as timer
#Clear output after training
import IPython
import IPython.display
#Normalization
from sklearn.preprocessing import MinMaxScaler
#Evaluate MSE
from sklearn.metrics import mean_squared_error
#plot numpy array
import matplotlib.pyplot as plt
#Keras: Open-Source deep-learning library 
from tensorflow import keras
#Building blocks of NN in Keras
from tensorflow.keras import layers
#EarlyStop to stop training early
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
#K-Means Clustering
from tslearn.clustering import TimeSeriesKMeans
import tensorflow as tf
#Path for model Checkpoint
import os
#Helper Class (Export Notebook as .py)
from windowgenerator import WindowGenerator
from federated_helper_functions import *
#Logging of debug, errors, info, warning, error
import logging
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)

#Data Analytics ---------------------------------------------------------------------------

#Define Path to datafile
user = 'rs1044' #'Jonas'
filename = '00Data_KIT_2019-01-01_to_2021_12_31.csv'
path = 'C:/Users/' + user + '/bwSyncShare/02Python Code/Data/' + filename

#1.1 Data collection & Visualization
#Read CSV file to pandas dataframe; encoding= 'unicode_escape': Decode from Latin-1 source code. Default UTF-8.
df = pd.read_csv(path, encoding= 'unicode_escape', index_col='Date')
df.index = pd.to_datetime(df.index)
df_feature = df.copy()
#Drop columns with insignificant weather features
df.drop(columns=['prcp', 'wdir','pres', 'coco','dwpt', 'wpgt','wspd'], axis=1, inplace=True)
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
#Display smart meter names and amount
smart_meter_names = df.columns[2:-2]
print(len(smart_meter_names))
#Convert Time String to seconds
date_time = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
timestamp_s = date_time.map(pd.Timestamp.timestamp)
#Sine cosine Scaling
hour = 60*60
df['hour sin'] = np.sin(timestamp_s * (2 * np.pi / hour))
df['hour cos'] = np.cos(timestamp_s * (2 * np.pi / hour))
df.drop('hour', axis=1, inplace=True)
daysinweek = 7*24*60*60
df['dayofweek sin'] = np.sin(timestamp_s * (2 * np.pi / daysinweek))
df['dayofweek cos'] = np.cos(timestamp_s * (2 * np.pi / daysinweek))
df.drop('dayofweek', axis=1, inplace=True)
#Min Max Sclaing
col_names = df.columns
features = df[col_names]
scaler = MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
df = pd.DataFrame(features, columns = col_names, index=df.index)

#K-menas Clustering
N_CLUSTERS=6
"""
#New Dataframe for clustering data without time and weather features
time_series_data = df.drop(['temp', 'rhum', 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos'], axis=1)
time_series_data.reset_index(inplace=True)
#Downsample Data to improve Clustering runtime
undersample_data = time_series_data.loc[np.linspace(time_series_data.index.min(),time_series_data.index.max(),3000).astype(int)]
undersample_data = undersample_data.reset_index().drop('index',axis=1)
#Performe K-means clustering with Dynamic Time Warping
data_array = np.array(undersample_data.T.drop('Date').values)
model = TimeSeriesKMeans(n_clusters=N_CLUSTERS, metric="dtw", max_iter=40)
model.fit(data_array)
y=model.predict(data_array)
x = undersample_data.Date
print("Clustering results: ", y)
"""
y=[2, 2, 4, 4, 1, 5, 1, 5, 1, 5, 5, 5, 5, 1, 3, 0, 3, 0, 5, 1, 3, 3, 3, 3, 5, 2, 2, 1, 3, 0, 0, 1, 2]
print("Clustering results: ", y)

# Create Datasets for the 33 clients and for 5 and 7 features
#ds_dict[smart_meter_names][0-5] 
#    -> 0:train_df_F7, 1: val_df_F7, 2: test_df_F7, 3: train_df_F5, 4: val_df_F5, 5: test_df_F5
ds_dict = {}
n = len(df)
for client in smart_meter_names:   
    train_df_F7 = df[0:int(n*0.7)][[client, 'temp', 'rhum', 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    val_df_F7 = df[int(n*0.7):int(n*0.9)][[client, 'temp', 'rhum', 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    test_df_F7 = df[int(n*0.9):][[client, 'temp', 'rhum', 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    
    train_df_F5 = df[0:int(n*0.7)][[client, 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    val_df_F5 = df[int(n*0.7):int(n*0.9)][[client, 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    test_df_F5 = df[int(n*0.9):][[client, 'hour sin', 'hour cos', 'dayofweek sin', 'dayofweek cos']]
    
    ds_dict[client] = [train_df_F7, val_df_F7, test_df_F7, train_df_F5, val_df_F5, test_df_F5]

#Initialize results
final_dict = {}
final_dict['Federated'] = {}
final_dict['Federated']['LSTM'] = {}
final_dict['Federated']['LSTM']['H12'] = {}
final_dict['Federated']['LSTM']['H12']['F5'] = {}
final_dict['Federated']['LSTM']['H12']['F7'] = {}
#----------------------------------------------
final_dict['Federated']['LSTM']['H24'] = {}
final_dict['Federated']['LSTM']['H24']['F5'] = {}
final_dict['Federated']['LSTM']['H24']['F7'] = {}

final_dict['Federated']['CNN'] = {}
final_dict['Federated']['CNN']['H12'] = {}
final_dict['Federated']['CNN']['H12']['F5'] = {}
final_dict['Federated']['CNN']['H12']['F7'] = {}
#----------------------------------------------
final_dict['Federated']['CNN']['H24'] = {}
final_dict['Federated']['CNN']['H24']['F5'] = {}
final_dict['Federated']['CNN']['H24']['F7'] = {}

final_dict['Federated']['Transformer'] = {}
final_dict['Federated']['Transformer']['H12'] = {}
final_dict['Federated']['Transformer']['H12']['F5'] = {}
final_dict['Federated']['Transformer']['H12']['F7'] = {}
#----------------------------------------------
final_dict['Federated']['Transformer']['H24'] = {}
final_dict['Federated']['Transformer']['H24']['F5'] = {}
final_dict['Federated']['Transformer']['H24']['F7'] = {}

#Set Hyperparameter ------------------------------------------------------------------------------
OUT_STEPS = [12, 24] #Next 12 or 24 hours
NUM_FEATURES = [5, 7] # [F_T, F_TW] load_value, hour sin, hour cos, dayofweek sin, dayofweek cos + (temp, rhum)
INPUT_STEPS = 24
INPUT_SHAPE = [(INPUT_STEPS, NUM_FEATURES[0]), (INPUT_STEPS, NUM_FEATURES[1])]

#All models
MAX_EPOCHS = 2

#LSTM
NUM_LSTM_LAYERS = 4
NUM_LSTM_CELLS = 32
NUM_LSTM_DENSE_LAYERS=1
NUM_LSTM_DENSE_UNITS = 32
LSTM_DROPOUT = 0.2

#CNN
CONV_WIDTH = 3
NUM_CNN_LAYERS = 4
NUM_CNN_FILTERS = 24
NUM_CNN_DENSE_LAYERS = 1
NUM_CNN_DENSE_UNITS = 32
CNN_DROPOUT = 0.2

#Federated Learning
comms_round = 20

#Windowing-----------------------------------------------------------------
#ds_dict[smart_meter_names][0-5] 
#    -> 0:train_df_F7, 1: val_df_F7, 2: test_df_F7, 3: train_df_F5, 4: val_df_F5, 5: test_df_F5

#windows_dict[cluster 0-5][client_i_smart_meter_names][0-3] 
#    -> 0:window_F5_H12 , 1:window_F5_H24 , 2:window_F7_H12 , 3:window_F7_H24
windows_dict = {k: {} for k in range(N_CLUSTERS)}

for i, client in enumerate(smart_meter_names):
    #window_F5_H12
    window_F5_H12 = WindowGenerator(
        input_width=INPUT_STEPS, label_width=OUT_STEPS[0], shift=OUT_STEPS[0], 
        train_df = ds_dict[client][3], val_df = ds_dict[client][4], test_df = ds_dict[client][5], label_columns=[client]
    )
    example_window = tf.stack([np.array(ds_dict[client][3][10100:10100+window_F5_H12.total_window_size]),
                               np.array(ds_dict[client][3][2000:2000+window_F5_H12.total_window_size]),
                               np.array(ds_dict[client][3][3000:3000+window_F5_H12.total_window_size])])
    example_inputs, example_labels = window_F5_H12.split_window(example_window)
    window_F5_H12.example = example_inputs, example_labels

    #window_F5_H24
    window_F5_H24 = WindowGenerator(
        input_width=INPUT_STEPS, label_width=OUT_STEPS[1], shift=OUT_STEPS[1], 
        train_df = ds_dict[client][3], val_df = ds_dict[client][4], test_df = ds_dict[client][5], label_columns=[client]
    )
    example_window = tf.stack([np.array(ds_dict[client][3][10100:10100+window_F5_H24.total_window_size]),
                               np.array(ds_dict[client][3][2000:2000+window_F5_H24.total_window_size]),
                               np.array(ds_dict[client][3][3000:3000+window_F5_H24.total_window_size])])
    example_inputs, example_labels = window_F5_H24.split_window(example_window)
    window_F5_H24.example = example_inputs, example_labels

    #window_F7_H12
    window_F7_H12 = WindowGenerator(
        input_width=INPUT_STEPS, label_width=OUT_STEPS[0], shift=OUT_STEPS[0], 
        train_df = ds_dict[client][0], val_df = ds_dict[client][1], test_df = ds_dict[client][2], label_columns=[client]
    )
    example_window = tf.stack([np.array(ds_dict[client][0][10100:10100+window_F7_H12.total_window_size]),
                               np.array(ds_dict[client][0][2000:2000+window_F7_H12.total_window_size]),
                               np.array(ds_dict[client][0][3000:3000+window_F7_H12.total_window_size])])
    example_inputs, example_labels = window_F7_H12.split_window(example_window)
    window_F7_H12.example = example_inputs, example_labels

    #window_F5_H24
    window_F7_H24 = WindowGenerator(
        input_width=INPUT_STEPS, label_width=OUT_STEPS[1], shift=OUT_STEPS[1], 
        train_df = ds_dict[client][0], val_df = ds_dict[client][1], test_df = ds_dict[client][2], label_columns=[client]
    )
    example_window = tf.stack([np.array(ds_dict[client][0][10100:10100+window_F7_H24.total_window_size]),
                               np.array(ds_dict[client][0][2000:2000+window_F7_H24.total_window_size]),
                               np.array(ds_dict[client][0][3000:3000+window_F7_H24.total_window_size])])
    example_inputs, example_labels = window_F7_H24.split_window(example_window)
    window_F7_H24.example = example_inputs, example_labels
    
    windows_dict[y[i]]['{}_{}_{}'.format('client', i+1, client)] = [window_F5_H12, window_F5_H24, window_F7_H12, window_F7_H24]
    
    
def compile_fit_set_weights(local_model, global_weights, window, client, client_names, model_type):
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,mode='min')
    local_model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.RootMeanSquaredError(), 
            tf.keras.metrics.MeanAbsolutePercentageError(),
            tf.keras.metrics.MeanAbsoluteError(),
        ]
    )
    #set local model weight to the weight of the global model
    local_model.set_weights(global_weights)
    #fit local model with client's data
    local_model.fit(window.train, epochs=MAX_EPOCHS, verbose=1, validation_data=window.val,
                      callbacks=[early_stopping, create_model_checkpoint(save_path=f"model_experiments/Federated/{local_model.name}/{client}"), timetaken]
                   )
    
    #scale the model weights and add to list        
    scaling_factor = weight_scalling_factor(window.train, client, client_names)
    scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
    
    if (model_type == 'LSTM'):
        scaled_local_weight_LSTM_list.append(scaled_weights)
    elif (model_type == 'CNN'):
        scaled_local_weight_CNN_list.append(scaled_weights)
    elif (model_type == 'Transformer'):
        scaled_local_weight_Transformer_list.append(scaled_weights)
    
    #clear session to free memory after each communication round
    K.clear_session()

# Set random seed for as reproducible results as possible
tf.random.set_seed(42)

#Federated Training ------------------------------------------------------
#Track memory usage
# importing the module
import tracemalloc

# starting the monitoring
tracemalloc.start()

try:
    ### Features 7, Horizon 24
    global_LSTM_model = []
    global_CNN_model = []
    global_Transformer_model = []

    for idx, cluster in enumerate(windows_dict):

        #Build Models
        global_LSTM_model.append(LSTM_Model().build(
            input_shape = INPUT_SHAPE[1], 
            num_LSTM_cells = NUM_LSTM_CELLS,
            num_LSTM_layers = NUM_LSTM_LAYERS,
            num_LSTM_dense_layers = NUM_LSTM_DENSE_LAYERS,
            num_LSTM_dense_units = NUM_LSTM_DENSE_UNITS,
            LSTM_dropout = LSTM_DROPOUT,
            output_steps = OUT_STEPS[1],
            num_features = NUM_FEATURES[1],
            model_name = 'Federated_LSTM_F7_H24'
        ))
        #CNN        
        global_CNN_model.append(CNN_Model().build(
            input_shape = INPUT_SHAPE[1], 
            conv_width = CONV_WIDTH,
            num_CNN_layers = NUM_CNN_LAYERS,
            num_CNN_filters = NUM_CNN_FILTERS,
            num_CNN_dense_layers = NUM_CNN_DENSE_LAYERS,
            num_CNN_dense_units = NUM_CNN_DENSE_UNITS,
            CNN_dropout = CNN_DROPOUT,
            output_steps = OUT_STEPS[1],
            num_features = NUM_FEATURES[1],
            model_name = 'Federated_CNN_F7_H24'
        ))
        #Transformer
        global_Transformer_model.append(Transformer_Model().build(
            input_shape = INPUT_SHAPE[1],
            output_steps = OUT_STEPS[1],
            num_features = NUM_FEATURES[1],
            model_name = 'Federated_Transformer_F7_H24'    
        ))

    #windows_dict[client_i_smart_meter_names][0-3] 
    #    -> 0:window_F5_H12 , 1:window_F5_H24 , 2:window_F7_H12 , 3:window_F7_H24

    #commence global training loop
    for idx_com, comm_round in enumerate(range(comms_round)):

        for idx, cluster in enumerate(windows_dict):
            IPython.display.clear_output()
            print("--------Federated Round---", idx_com+1, "/", comms_round, "---Cluster--", idx+1, "/5")
            # Get the global model's weights 
            global_LSTM_weights = global_LSTM_model[idx].get_weights()
            global_CNN_weights = global_CNN_model[idx].get_weights()
            global_Transformer_weights = global_Transformer_model[idx].get_weights()

            #initial list for local model weights after scalling
            scaled_local_weight_LSTM_list = list()
            scaled_local_weight_CNN_list = list()
            scaled_local_weight_Transformer_list = list()


            #Get names of clients within cluster
            client_names = list()
            for client in windows_dict[cluster]:
                client_names.append(client)

            for client in windows_dict[cluster].keys():

                #LSTM
                local_LSTM_model = LSTM_Model().build(
                    INPUT_SHAPE[1], NUM_LSTM_CELLS, NUM_LSTM_LAYERS, NUM_LSTM_DENSE_LAYERS, NUM_LSTM_DENSE_UNITS,
                    LSTM_DROPOUT, OUT_STEPS[1], NUM_FEATURES[1], 'Federated_local_LSTM_F7_H24'
                )
                compile_fit_set_weights(local_LSTM_model, global_LSTM_weights, windows_dict[cluster][client][3], client, client_names, 'LSTM')

                #CNN
                local_CNN_model = CNN_Model().build(
                    INPUT_SHAPE[1], CONV_WIDTH, NUM_CNN_LAYERS, NUM_CNN_FILTERS, NUM_CNN_DENSE_LAYERS, NUM_CNN_DENSE_UNITS,
                    CNN_DROPOUT, OUT_STEPS[1], NUM_FEATURES[1],'Federated_local_CNN_F7_H24'
                )    
                compile_fit_set_weights(local_CNN_model, global_CNN_weights, windows_dict[cluster][client][3], client, client_names, 'CNN')

                #Transformer
                local_Transformer_model = Transformer_Model().build(
                    INPUT_SHAPE[1],OUT_STEPS[1],NUM_FEATURES[1],'Federated_local_Transformer_F7_H24'    
                )
                compile_fit_set_weights(local_Transformer_model, global_Transformer_weights, windows_dict[cluster][client][3], client, client_names, 'Transformer')

            #to get the average over all the local model, we simply take the sum of the scaled weights
            average_weights_LSTM = sum_scaled_weights(scaled_local_weight_LSTM_list)
            average_weights_CNN = sum_scaled_weights(scaled_local_weight_CNN_list)
            average_weights_Transformer = sum_scaled_weights(scaled_local_weight_Transformer_list)
            #update global model 
            global_LSTM_model[idx].set_weights(average_weights_LSTM)
            global_CNN_model[idx].set_weights(average_weights_CNN)
            global_Transformer_model[idx].set_weights(average_weights_Transformer)

    #Evaluate Results
    forecasts_dict_LSTM_F7_H24 = {k: {} for k in range(N_CLUSTERS)}
    forecasts_dict_CNN_F7_H24 = {k: {} for k in range(N_CLUSTERS)}
    forecasts_dict_Transformer_F7_H24 = {k: {} for k in range(N_CLUSTERS)}

    for idx, cluster in enumerate(windows_dict):
        #Get names of clients within cluster
        client_names = list()
        for client in windows_dict[cluster]:
            client_names.append(client)

        for i, client in enumerate(windows_dict[cluster].keys()):
            IPython.display.clear_output()
            print("-------------Cluster----", cluster, "-----", client,"--------", i+1, "/", len(client_names))

            #LSTM
            model_evaluation_test = test_model(windows_dict[cluster][client][3], global_LSTM_model[idx], client, MAX_EPOCHS)
            #Save
            forecasts_dict_LSTM_F7_H24[cluster][client] = {
                'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
                'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
            }
            #CNN
            model_evaluation_test = test_model(windows_dict[cluster][client][3], global_CNN_model[idx], client, MAX_EPOCHS)
            #Save
            forecasts_dict_CNN_F7_H24[cluster][client] = {
                'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
                'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
            }    
            #Transformer
            model_evaluation_test = test_model(windows_dict[cluster][client][3], global_Transformer_model[idx], client, MAX_EPOCHS)
            #Save
            forecasts_dict_Transformer_F7_H24[cluster][client] = {
                'MSE':model_evaluation_test[0], 'RMSE':model_evaluation_test[1], 'MAPE':model_evaluation_test[2],
                'MAE':model_evaluation_test[3], 'Time':((timetaken.logs[-1][1]) / (timetaken.logs[-1][0]+1)) 
            }

    final_dict['Federated']['LSTM']['H24']['F7'] = forecasts_dict_LSTM_F7_H24
    final_dict['Federated']['CNN']['H24']['F7'] = forecasts_dict_CNN_F7_H24
    final_dict['Federated']['Transformer']['H24']['F7'] = forecasts_dict_Transformer_F7_H24

    import pickle 
    with open('Dictionaries_Results/Federated_resultsF5H12.pkl', 'wb') as f:
        pickle.dump(final_dict, f)
 
except Exception, e: 
    logger.error('Failed to upload to ftp: '+ str(e))

    
